In [11]:
def Find_stars_photometery(file_path, file, new_path, N_bits, binned, number_of_stars,FWHM_x):
    
    from astropy.stats import sigma_clipped_stats
    from photutils.detection import DAOStarFinder
    import numpy as np
    from photutils.aperture import ApertureStats
    from photutils.aperture import CircularAperture, CircularAnnulus
    from photutils.aperture import aperture_photometry
    from astropy.io import fits

 

        
    threshold_for_DAO = 5.0
    
    hdul = fits.open(file_path / file)    
    data = hdul[0].data
    header_data = hdul[0].header

 

    arcsec_pix = header_data['SCALE']
    exposure_time = header_data['EXPTIME']

 

    
    # Our usual sigma for a focused star is about 6 arc-seconds, need to convert to pixels
    FWHM = 6 /  arcsec_pix

 

    mean, median, std = sigma_clipped_stats(data, sigma=3.0)

 

        # (1) Use the first guesses of FWHM etc. to find "1st draft" stars

 

    daofind = DAOStarFinder(fwhm=FWHM, threshold=threshold_for_DAO*std, brightest=5,exclude_border=True)  
    sources = daofind(data - median)  
    for col in sources.colnames:  
        sources[col].info.format = '%.8g'  # for consistent table output
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))

 

    print('Done 1st draft find, found '+ str(len(positions))+' stars')

 

        # (2) Run aperature photometry on the "1st draft" stars
        # Note from VS thesis the aperature is about 5*FWHM to ensure all star is measured

 

    aperture = CircularAperture(positions, r=FWHM*2)
    aperstats = ApertureStats(data - median, aperture) #So that 2D Gaussian is sensible
    new_fwhm = aperstats.fwhm.value
        #print(new_fwhm)
    new_FWHM = np.nanmean(new_fwhm)

 

    print('Done 1st draft photometry')
    
    
    print('Old FWHM = %.2f pixs' % FWHM)
    FWHM_Arc = FWHM*arcsec_pix
    print('Old FWHM = %.2f arcsec' % FWHM_Arc) 
    
    print('New FWHM = %.2f pixs' % new_FWHM)
    
    FWHM_Arc = new_FWHM*arcsec_pix
    print('New FWHM = %.2f arcsec' % FWHM_Arc)
        # (3) Update the FWHM and re-run star finder

 

    FWHM = new_FWHM

 

    daofind = DAOStarFinder(fwhm=FWHM, threshold=threshold_for_DAO*std, brightest=number_of_stars,exclude_border=True)  
    sources = daofind(data - median)  
    for col in sources.colnames:  
        sources[col].info.format = '%.8g'  # for consistent table output
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))

 

    print('Done 2nd round find, found '+ str(len(positions))+' stars')

 

        # (4) Re-run aperature photometery with the new star list.
        # Set aperature to be 3*FWHM and anulus to have some area.

 

    r_value     = FWHM*FWHM_x
    # To get an outer annulus with 4x the area of the circular aperture, use this. Some way to ensuring tte uncertainty in the
    # mean or median sky is as low as we can get, as the spread of the sky in the actual aperture is always going to be the limiting facotr.
    # Excepet, if we add in to much annulus this will make our errors larger as we add in more and more read noise. Oh bugger.
    r_out_value = r_value*np.sqrt(5)

 

    ANNULUS_aperture = CircularAnnulus(positions, r_in=r_value, r_out=r_out_value)
    CIRCULAR_aperture = CircularAperture(positions, r=r_value)

 

    ANNU_stats  = ApertureStats(data, ANNULUS_aperture)
    CIRC_stats = ApertureStats(data, CIRCULAR_aperture)

 

    phot_table = aperture_photometry(data, CIRCULAR_aperture)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output
    df_photometry = phot_table.to_pandas()

 


    df_photometry['Circ_sum_flux']      = CIRC_stats.sum

 

 

    #        df_photometry = pd.DataFrame(CIRC_stats.sum,columns=['Circ_sum_flux'])

 

    df_photometry['Circ_area']          = CIRC_stats.sum_aper_area.value
    df_photometry['Max_Flux']           = CIRC_stats.max

 

    df_photometry['FWHM_form_aperature'] = FWHM

 

    df_photometry['Annu_sum_flux']      = ANNU_stats.sum
    df_photometry['Annu_area']          = ANNU_stats.sum_aper_area.value
    df_photometry['Annu_pix_flux_median'] = ANNU_stats.median

 

    df_photometry['Circ_sum_flux_minus_back'] = \
        df_photometry['Circ_sum_flux'] - df_photometry['Annu_pix_flux_median'] * df_photometry['Circ_area']

 

    df_photometry.sort_values(by=['Circ_sum_flux_minus_back'],ascending=False,inplace=True,ignore_index=True)

 

    df_photometry['Apperature_radius_pixels'] = r_value
    df_photometry['Anulus_in_radius_pixels'] = r_value
    df_photometry['Anulus_out_radius_pixels'] = r_out_value

 

    saturation_limit = 2**(N_bits) / exposure_time
        ##########
    if binned == 1:
        saturation_limit = saturation_limit * 4
        ##########
    saturation_limit = saturation_limit * 0.95
    df_photometry['Saturated'] = df_photometry['Max_Flux'] > saturation_limit
    


    df_photometry['Number_FWHM'] = FWHM_x

 

    df_photometry['Exposure_sec'] = exposure_time

 

    # Compute uncertainty on the source flux

 

    Er1 = df_photometry['Circ_sum_flux']/df_photometry['Exposure_sec']
    Er2 = (df_photometry['Circ_area'] * df_photometry['Circ_area']  )/  (df_photometry['Exposure_sec']*df_photometry['Annu_area']  )
    Er3 = 2*df_photometry['Circ_area']* 1.5**2 / df_photometry['Exposure_sec']**2

 

    if binned == 1:
        Er3 = 2*df_photometry['Circ_area']*4* 1.5**2 / df_photometry['Exposure_sec']**2

 


    Er = np.sqrt(Er1+Er2+Er3)

 

    df_photometry['Circ_sum_flux_minus_back_ERROR'] = Er

 

    mag_error = -2.5 * Er / (df_photometry['Circ_sum_flux_minus_back'] * np.log(10))

 

    df_photometry['Source_Mag'] = -2.5*np.log10(df_photometry['Circ_sum_flux_minus_back'])
    df_photometry['Source_Mag_Error'] = mag_error
    
    df_photometry['Corrected_Mag'] = df_photometry['Source_Mag'] + 2.5 * np.log10(df_photometry['Exposure_sec'])

 

    air_mass = header_data['AIRMASS']
    azimuth =  header_data['OBJCTAZ']
    altitude = header_data['OBJCTALT']
    obs_time     = header_data['DATE-OBS']
    exp_time     = header_data['EXPTIME ']
    filter_colour = header_data['FILTER  ']

 


    df_photometry['Air mass'] = air_mass
    df_photometry['Azimuth'] = azimuth
    df_photometry['Altitude'] = altitude
    df_photometry['Date Observation'] = obs_time
    df_photometry['Exposure time /s '] = exp_time
    df_photometry['Filter'] = filter_colour
    df_photometry['File'] = file
    
    #remove stars that are too dim, or negative!
    df_photometry.drop(df_photometry[df_photometry['Circ_sum_flux_minus_back'] <= 100].index, inplace = True)

 

    new_file_name = file.replace('.fits', '_photometery.csv')

 

    print('Done 2nd round photometry')
    print(new_file_name)
    df_photometry.to_csv(new_path / new_file_name)
    
    return df_photometry

In [22]:
from astropy.nddata import CCDData
import ccdproc as ccdp
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

import pandas as pd
from astroquery.simbad import Simbad
from astroquery.astrometry_net import AstrometryNet
from astropy.wcs import WCS
from astropy import coordinates
import astropy.units as u

In [17]:
#name strings for fits files and csv files
fits = '.fits'
csv = '_photometery.csv'

#filter specific file names
luminance = 'star_Light_Luminance_5_secs_001_reduced'
red = 'star_Light_Red_18_secs_001_reduced'
green = 'star_Light_Green_18_secs_001_reduced'
blue = 'star_Light_Blue_18_secs_001_reduced'

#set current filter
ref_image_name = luminance + fits
ref_csv_name = luminance + csv
current_filter = 'Luminance'

#define path for csv files to be written to
csv_path = Path(f'./RZ Cep/All_RZ_Cep/2022_12_07/RZ_Cep/CSVs/Unmatched Stars/{current_filter}')

#set the path to the images we are looking at
current_raw_path = f'./RZ Cep/All_RZ_Cep/2022_12_07/RZ_Cep/Lights/{current_filter} Reduced/'
#create a list of files in the directory (file) indicated by the path, just in case some sub folders exist or files that are
#not of the FITS type.
file_list = [f for f in os.listdir(current_raw_path) if f.endswith(".fits")]

#convert the path from a string into a Path object that can be put into functions
raw_path = Path(current_raw_path)
#read in the images
raw_ims = ccdp.ImageFileCollection(raw_path, filenames=file_list)

In [13]:
for file_name in file_list:
    df_phot = Find_stars_photometery(raw_path, file_name, csv_path, 16, 0, 150, 3.5)

Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 12.52 pixs
New FWHM = 6.14 arcsec
Done 2nd round find, found 150 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_001_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.41 pixs
New FWHM = 5.10 arcsec
Done 2nd round find, found 150 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_002_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.53 pixs
New FWHM = 5.16 arcsec
Done 2nd round find, found 150 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_003_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.96 pixs
New FWHM = 5.37 arcsec
Done 2nd round find, found 150 stars
Do

C:\ProgramData\Anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Done 2nd round photometry
star_Light_Luminance_5_secs_012_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.79 pixs
New FWHM = 5.29 arcsec
Done 2nd round find, found 150 stars


C:\ProgramData\Anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Done 2nd round photometry
star_Light_Luminance_5_secs_013_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.62 pixs
New FWHM = 5.21 arcsec
Done 2nd round find, found 150 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_014_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.09 pixs
New FWHM = 4.94 arcsec
Done 2nd round find, found 150 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_015_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 10.36 pixs
New FWHM = 5.08 arcsec
Done 2nd round find, found 150 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_016_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
Ne

...

Done 2nd round photometry
star_Light_Luminance_5_secs_154_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 15.98 pixs
New FWHM = 7.83 arcsec
Done 2nd round find, found 63 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_155_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 14.44 pixs
New FWHM = 7.07 arcsec
Done 2nd round find, found 92 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_156_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 14.72 pixs
New FWHM = 7.21 arcsec
Done 2nd round find, found 104 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_157_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New 

C:\ProgramData\Anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Done 2nd round photometry
star_Light_Luminance_5_secs_158_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 14.59 pixs
New FWHM = 7.15 arcsec
Done 2nd round find, found 98 stars


C:\ProgramData\Anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Done 2nd round photometry
star_Light_Luminance_5_secs_159_reduced_photometery.csv
Done 1st draft find, found 5 stars
Done 1st draft photometry
Old FWHM = 12.25 pixs
Old FWHM = 6.00 arcsec
New FWHM = 15.52 pixs
New FWHM = 7.61 arcsec
Done 2nd round find, found 107 stars
Done 2nd round photometry
star_Light_Luminance_5_secs_160_reduced_photometery.csv


In [23]:
#read in the original image and the csv associated with it
raw_im = CCDData.read(raw_path / ref_image_name)
df_ref = pd.read_csv(csv_path / ref_csv_name, delimiter=',')

#get all info Simbad has on our main star
#set up Simbad search
customSimbad = Simbad()
#add fields to output
customSimbad.add_votable_fields('otype','flux(U)','flux(B)','flux(V)','flux(R)','flux(I)','flux(G)','flux(J)','flux(H)',
                                'flux(K)','flux(u)','flux(g)','flux(r)','flux(i)','flux(z)','ra(deg)','dec(deg)','ids')

#search for object and its properties, saving to a dataframe
RZ_Cep = customSimbad.query_object('RZ Cep')
df_RZ_Cep = RZ_Cep.to_pandas()

#find star position in degrees
RZ_Cep_ra = df_RZ_Cep['RA_deg'][0]
RZ_Cep_dec = df_RZ_Cep['DEC_deg'][0]
print(RZ_Cep_ra, RZ_Cep_dec)

#set up Astrometry.net solver, which compares image to the sky to find exact sky position
ast = AstrometryNet()
ast.api_key = 'uktlwsvagbqfbbwl'

#define image size
image_width = raw_im.header['NAXIS1']
image_height = raw_im.header['NAXIS2']
print(image_width, image_height)

#use astrometry.net to match the image data to the sky and find correct coordinates 
wcs_header = ast.solve_from_source_list(df_ref['xcenter'], df_ref['ycenter'], #input star positions to be compared
                                        image_width, image_height, #image size for matching to the sky
                                        solve_timeout=180, #max time for the solving to be done in
                                        #scale_type='ul',
                                        scale_est = raw_im.header['SCALE'], #check header to see if scaling is required
                                        scale_err = 5, #estimate scale error
                                        #scale_lower = raw_im.header['SCALE'] * 0.9,
                                        #scale_upper = raw_im.header['SCALE'] * 1.1,
                                        scale_units = 'arcsecperpix', #units of scaling
                                        crpix_center=True, #force reference point to be centre of image
                                        center_ra = RZ_Cep_ra, #set image centre as star centre
                                        center_dec = RZ_Cep_dec, 
                                        radius = (raw_im.header['SCALE'] * image_width)/3600 #scale to arcseconds
                                       )

#convert extracted information into a world coordinate system for the image, which is added to the image header
w = WCS(wcs_header)
print(w)
raw_im.header.update(w.to_header(), 'relax')

#change reference point in header to sentre of star rather then image center
raw_im.header['CRPIX1'] = df_ref['xcenter'][0]
raw_im.header['CRPIX2'] = df_ref['ycenter'][0]

#convert pixel values to ra and dec
sky_all = w.pixel_to_world(df_ref['xcenter'], df_ref['ycenter'])
#append to original dataframe containing pixel position values, to be saved in csv file
df_ref['ra_from_astrometry_deg'] = sky_all.ra.degree
df_ref['dec_from_astrometry_deg'] = sky_all.dec.degree

#save all stars, their names, ra and dec, Simbad properties within a certain radius of the centre to a dataframe
#set up coordinates associated with 
c = coordinates.SkyCoord(sky_all)
#ensures all stars recieve their id, as a smaller radius means stars close to the edge of the image will not get a proper id
r = 5 * u.arcminute
result_table = customSimbad.query_region(c, radius=r)
df_results = result_table.to_pandas()

#add id to csv file and overwrite it
df_ref['star_id'] = df_results['MAIN_ID']
df_ref = df_ref.drop('Unnamed: 0', axis=1)
df_ref.to_csv(csv_path / ref_csv_name)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.


339.80490354369 64.85850316231
4656 3520
Solving.......

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN-SIP'  'DEC--TAN-SIP'  
CRVAL : 339.798299443  64.8654358589  
CRPIX : 2328.5  1760.5  
CD1_1 CD1_2  : -2.61829006281e-05  0.000132296778495  
CD2_1 CD2_2  : -0.000132256159789  -2.62222255078e-05  
NAXIS : 0  0


In [24]:
df_ref

,id,xcenter,ycenter,aperture_sum,Circ_sum_flux,Circ_area,Max_Flux,FWHM_form_aperature,Annu_sum_flux,Annu_area,...,Air mass,Azimuth,Altitude,Date Observation,Exposure time /s,Filter,File,ra_from_astrometry_deg,dec_from_astrometry_deg,star_id
0,1,2365.708025,1793.419559,3.290334e+06,3.290334e+06,6034.979809,15961.239946,12.522612,6.079980e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.806508,64.859493,V* RZ Cep
1,3,269.974547,3162.321475,3.010081e+06,3.010081e+06,6034.979809,10825.663962,12.522612,6.151356e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,340.367184,65.099790,TYC 4273-435-1
2,2,3205.683885,1082.783297,2.632768e+06,2.632768e+06,6034.979809,11148.792813,12.522612,6.066250e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.534389,64.766822,TYC 4273-1351-1
3,5,4047.766258,2902.885500,2.345032e+06,2.345032e+06,6034.979809,6448.922897,12.522612,6.089593e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,340.046171,64.607723,BD+64 1700
4,4,2152.543210,68.043055,2.154321e+06,2.154321e+06,6034.979809,7942.553174,12.522612,5.316420e+06,21293.203136,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.281240,64.932071,TYC 4273-876-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,119,622.737819,1356.725666,1.546027e+06,1.546027e+06,6034.979809,670.386805,12.522612,6.145631e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.777780,65.101458,IRAS 22404+6450
146,129,448.276603,664.687929,1.549791e+06,1.549791e+06,6034.979809,624.225739,12.522612,6.139040e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.570942,65.142498,NVSS J224218+650345
147,136,1960.918502,2060.147268,1.523948e+06,1.523948e+06,6034.979809,571.173325,12.522612,6.039998e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.914723,64.905993,TYC 4273-876-1
148,144,3440.007490,323.267846,1.533966e+06,1.533966e+06,6034.979809,567.370663,12.522612,6.110122e+06,24139.919236,...,1.08652,320.1292,66.96391,2022-12-07T20:08:35.241,5.0,Luminance,star_Light_Luminance_5_secs_001_reduced.fits,339.284653,64.755139,2MASS J22420892+6508103
